# Getting started with MovingPandas

<img align="right" src="https://anitagraser.github.io/movingpandas/assets/img/movingpandas.png">

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/anitagraser/movingpandas-examples/main?filepath=1-tutorials/1-getting-started.ipynb)
[![IPYNB](https://img.shields.io/badge/view-ipynb-hotpink)](https://github.com/anitagraser/movingpandas-examples/blob/main/1-tutorials/1-getting-started.ipynb)
[![HTML](https://img.shields.io/badge/view-html-green)](https://anitagraser.github.io/movingpandas-website/1-tutorials/1-getting-started.html)

MovingPandas provides a trajectory datatype based on GeoPandas.
The project home is at https://github.com/anitagraser/movingpandas

The documentation is available at https://movingpandas.readthedocs.io/en/master/

In [ ]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame, read_file
import shapely as shp
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
import movingpandas as mpd
import hvplot.pandas 

import warnings
warnings.filterwarnings('ignore')

mpd.show_versions()

## Creating a trajectory from scratch

Trajectory objects consist of a trajectory ID and a GeoPandas GeoDataFrame with a DatetimeIndex. The data frame therefore represents the trajectory data as a Pandas time series with associated point locations (and optional further attributes).

Let's create a small toy trajectory to see how this works:

In [ ]:
df = pd.DataFrame([
  {'geometry':Point(0,0), 't':datetime(2018,1,1,12,0,0)},
  {'geometry':Point(6,0), 't':datetime(2018,1,1,12,6,0)},
  {'geometry':Point(6,6), 't':datetime(2018,1,1,12,10,0)},
  {'geometry':Point(9,9), 't':datetime(2018,1,1,12,15,0)}
]).set_index('t')
gdf = GeoDataFrame(df, crs=31256)
toy_traj = mpd.Trajectory(gdf, 1)
toy_traj

In [ ]:
toy_traj.plot()

We can also access the trajectory's GeoDataFrame:

In [ ]:
toy_traj.df

In [ ]:
toy_traj.df.plot()

## Loading trajectories from CSV

In [ ]:
df = pd.read_csv('../data/geolife_small.csv', delimiter=';')
df

In [ ]:
traj_collection = mpd.TrajectoryCollection(df, 'trajectory_id', t='t', x='X', y='Y')
print(traj_collection)

In [ ]:
traj_collection.plot(column='trajectory_id', legend=True, figsize=(9,5))

## Loading trajectories from a GeoPackage

In [ ]:
gdf = read_file('../data/geolife_small.gpkg')
gdf

After reading the trajectory point data from file, we want to construct the trajectories.

In [ ]:
traj_collection = mpd.TrajectoryCollection(gdf, 'trajectory_id', t='t')
print(traj_collection)

In [ ]:
traj_collection.plot(column='trajectory_id', legend=True, figsize=(9,5))

In [ ]:
traj_collection.plot()

### Accessing individual trajectories

In [ ]:
my_traj = traj_collection.trajectories[1]
print(my_traj)

In [ ]:
my_traj.plot(linewidth=5, capstyle='round', figsize=(9,3))

To visualize trajectories in their geographical context, we can also create interactive plots with basemaps:

In [ ]:
my_traj.hvplot(width=500, height=300, line_width=7.0, tiles='OSM')

## Creating a TrajectoryCollection from scratch

In [ ]:
from shapely.geometry import Point
import geopandas as gpd
import movingpandas as mpd
import pandas as pd
from datetime import timedelta

df = pd.DataFrame(
    {
        "t": pd.date_range("2020-01-01", periods=5, freq="min"),
        "trajectory_id": [1, 1, 2, 2, 2],
        "geometry": [Point(0, 0), Point(0, 1), Point(0, 2), Point(0, 3), Point(0, 4)],
    }
)
gdf = gpd.GeoDataFrame(df, crs=31256)
trajc = mpd.TrajectoryCollection(gdf, traj_id_col='trajectory_id', t='t')
trajc

In [ ]:
min_duration = timedelta(minutes=1)
trajc.trajectories = [traj for traj in trajc if traj.get_duration() > min_duration]
trajc